In [1]:
import gym
import numpy as np
from time import sleep

In [2]:
# e = gym.make('CartPole-v0')
# e.render()

In [3]:
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
env.reset()
env.render()
print("Action space {}".format(env.action_space))
print("State space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action space Discrete(6)
State space Discrete(500)


In [5]:
state = env.encode(3,1,2,0)
print("State: ",state)
env.s = state
env.render()

State:  328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [6]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

So env.step(action) -> returns:
   
    observation: obseration of the enviroments k/as states
    reward: action was beneficial or not
    done: if we have successfully picked up and droped off a passenger which called as epsiode
    info: additional info such as latency or performance for debugging purposes
    

In [7]:
env.s = 328
epochs = 0
penalties, reward = 0, 0
frames = [] #animation 

done  = False
while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    if reward == -10:
        penalties += 1
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action' : action,
        'reward' : reward
    })
    
    epochs += 1
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 7680
Penalties incurred: 2491


In [8]:
from IPython.display import clear_output
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 7680
State: 0
Action: 5
Reward: 20


In [9]:
env.P[328] # so it will tell the [action : {probability,next state,reward,done}]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [10]:
q_table = np.zeros([env.observation_space.n,env.action_space.n]) #to create the q-table as initial elements are zeros

In [11]:
import random
# from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# So epsilon is used when when we are selecting specific actions based on the Q values we already have values lies between [0,1]
#so if we select a pure greedy method then epsilon wiil be 0

# For plotting metrics
all_epochs = [] #used for calculating the number of times that learning algorithm work for entire dataset
all_penalties = [] # used for calculating how many time it drop the passenger at wrong location

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max) #updating the Q-value 
        
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.



In [12]:
q_table[328]

array([ -2.39573913,  -2.27325184,  -2.40580569,  -2.35761308,
       -10.95407572, -10.99334683])

In [14]:
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.4
Average penalties per episode: 0.0
